In [1]:
import h5py
import scipy.io as io
import PIL.Image as Image
import numpy as np
import os
import glob
from matplotlib import pyplot as plt
from scipy.ndimage.filters import gaussian_filter 
import scipy
import json
import torchvision.transforms.functional as F
from matplotlib import cm as CM
from image import *
from model import CSRNet
import torch
%matplotlib inline


Bad key "text.kerning_factor" on line 4 in
/home/sachi/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
from torchvision import datasets, transforms
transform=transforms.Compose([
                       transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
                   ])

In [3]:
root = '/home/sachi/Downloads/ShanghaiTech_Crowd_Counting_Dataset/'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [23]:
#now generate the ShanghaiA's ground truth
part_A_train = os.path.join(root,'part_A_final/train_data','images')
part_A_test = os.path.join(root,'part_A_final/test_data','images')
part_B_train = os.path.join(root,'part_B_final/train_data','images')
part_B_test = os.path.join(root,'part_B_final/test_data','images')
path_sets = [part_A_test]

In [24]:
img_paths = []
for path in path_sets:
    for img_path in glob.glob(os.path.join(path, '*.jpg')):
        img_paths.append(img_path)

In [25]:
checkpoint = torch.load('/home/sachi/Downloads/0model_best.pth.tar')

model = CSRNet(load_weights = True).to(device)

In [26]:
checkpoint = torch.load('/home/sachi/Downloads/0model_best.pth.tar')

In [27]:
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [30]:
mae = 0
img = 255.0 * F.to_tensor(Image.open(img_paths[1]).convert('RGB'))

img[0,:,:]=img[0,:,:]-92.8207477031
img[1,:,:]=img[1,:,:]-95.2757037428
img[2,:,:]=img[2,:,:]-104.877445883
img = img.to(device)

output = model(img.unsqueeze(0)).detach().cpu().sum().numpy()
print(output)

4382.1895


In [10]:
mae = 0
for i in range(len(img_paths)):
    img = 255.0 * F.to_tensor(Image.open(img_paths[i]).convert('RGB'))

    img[0,:,:]=img[0,:,:]-92.8207477031
    img[1,:,:]=img[1,:,:]-95.2757037428
    img[2,:,:]=img[2,:,:]-104.877445883
    img = img.to(device)
    #img = transform(Image.open(img_paths[i]).convert('RGB')).cuda()
    gt_file = h5py.File(img_paths[i].replace('.jpg','.h5').replace('images','ground_truth'),'r')
    groundtruth = np.asarray(gt_file['density'])
    output = model(img.unsqueeze(0))
    mae += abs(output.detach().cpu().sum().numpy()-np.sum(groundtruth))
    print (i,mae)
print (mae/len(img_paths))

OSError: Unable to open file (unable to open file: name = '/home/sachi/Downloads/ShanghaiTech_Crowd_Counting_Dataset/part_A_final/test_data/ground_truth/IMG_60.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)